In [ ]:
import re
from lxml import etree

def contar_palabras(texto):
    """Cuenta el número de palabras en una cadena de texto."""
    return len(re.findall(r'\b\w+\b', texto)) if texto else 0

def eliminar_enlaces(texto):
    """Elimina URLs del texto."""
    url_pattern = r'http\S+|www\.\S+'
    return re.sub(url_pattern, '', texto)

def eliminar_nonsense(texto):
    """Elimina patrones no deseados como timestamps, DOIs, ISBNs, filenames con extensiones y avisos de copyright."""
    # Eliminar timestamps como 2019-07-03T03:21:42.840889Z
    timestamp_pattern = fr'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}(?:\.\d+)?Z?'
    texto = re.sub(timestamp_pattern, '', texto)
    
    # Eliminar DOIs, con o sin 'doi:'
    doi_pattern = r'\b(?:doi:)?10\.\d{4,9}/[-._;()/:A-Z0-9]+\b'
    texto = re.sub(doi_pattern, '', texto, flags=re.IGNORECASE)
    
    # Eliminar ISBNs, con o sin 'urn:isbn:'
    isbn_pattern = r'\b(?:urn:isbn:)?(?:97[89]-)?\d{1,5}-\d{2,7}-\d{2,7}-\d\b'
    texto = re.sub(isbn_pattern, '', texto, flags=re.IGNORECASE)
    
    # Eliminar códigos similares como "B978-0-08-100596-5.22063-5", "10.1016/C2016-1-03722-7"
    other_patterns = r'\b[A-Z]?\d{4}-\d{2}-\d{2,}-\d{1,}\b'
    texto = re.sub(other_patterns, '', texto)
    
    # Eliminar filenames con extensiones (e.g., 3-s2.0-B9780081026717107018-f701-01-9780081026717.sml)
    filename_pattern = r'\b\w+\.(?:png|jpg|jpeg|gif|bmp|pdf|docx?|xlsx?|sml)\b'
    texto = re.sub(filename_pattern, '', texto, flags=re.IGNORECASE)
    
    # Eliminar avisos de copyright
    copyright_pattern = r'\bCopyright\s?©\b.*'
    texto = re.sub(copyright_pattern, '', texto, flags=re.IGNORECASE)
    
    # Eliminar múltiples espacios
    texto = re.sub(r'\s+', ' ', texto)
    
    return texto.strip()

def eliminar_referencias(texto):
    """Elimina líneas que contienen 'et al' y tienen menos de 7 palabras."""
    if 'et al' in texto.lower() and contar_palabras(texto) < 7:
        return None
    return texto

def eliminar_logo_copyright(texto):
    """Elimina líneas que contienen símbolos de copyright."""
    if re.search(r'\(c\)|©', texto):
        return None
    return texto

def tiene_demasiados_caracteres_especiales(texto, porcentaje=0.1):
    """Elimina líneas donde más del porcentaje de caracteres son especiales."""
    if not texto:
        return False
    especiales = re.findall(r'[.,()\-]', texto)
    return len(especiales) / len(texto) > porcentaje

# Ruta del archivo XML
archivo_entrada = 'articulos/10.1016_B978-0-08-100596-5.22063-5.xml'
archivo_salida_all = 'textos_filtrados_all.txt'
archivo_salida_20 = 'textos_filtrados_20_split.txt'

# Parsear el XML
tree = etree.parse(archivo_entrada)
root = tree.getroot()

# Conjunto para almacenar textos filtrados y eliminar duplicados
textos_filtrados = set()
textos_filtrados_20_split = set()

# Iterar sobre todos los elementos, excluyendo los que contienen 'author' en su etiqueta
for elem in root.iter():
    if 'author' in elem.tag.lower():
        continue  # Ignorar elementos relacionados con autores
    texto = elem.text
    if texto:
        # Eliminar enlaces
        texto_sin_enlaces = eliminar_enlaces(texto.strip())
        # Eliminar patrones no deseados
        texto_limpio = eliminar_nonsense(texto_sin_enlaces)
        # Eliminar referencias no deseadas
        texto_final = eliminar_referencias(texto_limpio)
        if texto_final:
            # Eliminar líneas con demasiados caracteres especiales
            if not tiene_demasiados_caracteres_especiales(texto_final):
                # Eliminar líneas que contienen copyright
                texto_final = eliminar_logo_copyright(texto_final)
                if texto_final:
                    num_palabras = contar_palabras(texto_final)
                    # Filtrar textos con 5 palabras o más
                    if num_palabras >= 5:
                        textos_filtrados.add(texto_final)
                        # Para textos con más de 20 palabras, realizar el split
                        if num_palabras > 20:
                            # Dividir por puntos
                            oraciones = texto_final.split('.')
                            for oracion in oraciones:
                                oracion = oracion.strip()
                                if contar_palabras(oracion) > 10:
                                    textos_filtrados_20_split.add(oracion)

# Guardar todos los textos filtrados en 'textos_filtrados_all.txt'
with open(archivo_salida_all, 'w', encoding='utf-8') as f_all:
    for linea in textos_filtrados:
        f_all.write(linea + '\n')

# Guardar las oraciones filtradas en 'textos_filtrados_20_split.txt'
with open(archivo_salida_20, 'w', encoding='utf-8') as f_20:
    for linea in textos_filtrados_20_split:
        f_20.write(linea + '\n')

# Contar las líneas
total_all = len(textos_filtrados)
total_20 = len(textos_filtrados_20_split)

print(f"Se han extraído {total_all} líneas con 5 palabras o más a '{archivo_salida_all}'")
print(f"Se han extraído {total_20} oraciones con más de 10 palabras a '{archivo_salida_20}'")


Se han extraído 33 líneas con 5 palabras o más a 'textos_filtrados_all.txt'
Se han extraído 41 oraciones con más de 10 palabras a 'textos_filtrados_20_split.txt'


     ---------------------------------------- 3.8/3.8 MB 12.1 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
